In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
import pandas as pd

# 데이터 불러오기
titanic_cleaned = pd.read_csv('titanic_cleaned.csv')

# 데이터 확인
titanic_cleaned.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,False,False,True,False,False,False,False,True
1,1,1,1,38.0,1,0,71.2833,False,False,False,True,False,True,False,False
2,1,3,1,26.0,0,0,7.9250,False,True,False,False,False,False,False,True
3,1,1,1,35.0,1,0,53.1000,False,False,False,True,False,False,False,True
4,0,3,0,35.0,0,0,8.0500,False,False,True,False,False,False,False,True


# 데이터 분할

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

# 특성과 타겟 변수 분리
X = titanic_cleaned.drop('Survived', axis=1)
y = titanic_cleaned['Survived']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Stratified K-Fold 적용
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# KFold 적용
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, val_index in skf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

# 결정트리 학습 및 성능 평가

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# 디시젼 트리 모델 생성
dt_model = DecisionTreeClassifier(random_state=42)

# 모델 학습
dt_model.fit(X_train, y_train)

# 예측
y_pred = dt_model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Decision Tree Accuracy: {round(accuracy, 4)}')
print(f'Decision Tree Recall: {round(recall, 4)}')
print(f'Decision Tree Precision: {round(precision, 4)}')
print(f'Decision Tree F1 Score: {round(f1, 4)}')


Decision Tree Accuracy: 0.8156
Decision Tree Recall: 0.7246
Decision Tree Precision: 0.7812
Decision Tree F1 Score: 0.7519


---

# 결정트리 하이퍼 파라미터 튜닝

- max_depth
  - 디시젼 트리의 최대 깊이
  - 값의 범위는 [None, 1, 2, 3, ...]
  - 깊이가 깊을수록 모델이 더 복잡해지고 과적합될 가능성이 높아짐
- min_samples_split
  - 내부 노드를 분할하는 데 필요한 최소 샘플 수
  - 값의 범위는 [2, 3, 4, ...]
  - 값이 클수록 트리가 덜 복잡해짐
- min_samples_leaf
  - 리프 노드에 있어야 하는 최소 샘플 수
  - 값의 범위는 [1, 2, 3, ...]
  - 값이 클수록 트리가 덜 복잡해짐

디시전 트리(Decision Tree)에는 반복 횟수를 조절하는 하이퍼파라미터가 없음   
이유는 트리의 학습 과정이 반복(iterative)이 아니라 재귀(recursive)로 이루어지기 때문.

[디시전 트리의 학습 원리]
- 분할 방식
  - 데이터를 재귀적으로 분할하며 학습이 진행됨.
  - 각 단계에서 특정 기준(예: Gini 계수, 엔트로피)을 최적화하는 분할을 수행.
- 학습의 종결
  - 반복적인 학습 없이 조건에 따라 종료.
  - 트리가 최대 깊이(max_depth)에 도달.
  - 노드 내 샘플 수가 min_samples_split 이하.
  - 노드가 순수(pure)해 더 이상 분할할 필요가 없음.
- 결과
  - 트리는 각 단계에서 결정적으로 분할을 선택하며 모든 학습이 완료되면 더 이상 수정하거나 반복할 필요가 없음.

In [5]:
max_depth_values = [3, 5, 7, 10, 15, 20, None]
results = []

for depth in max_depth_values:
    dt_model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    dt_model.fit(X_train, y_train)
    y_pred = dt_model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results.append({
        'max_depth': depth,
        'accuracy': round(accuracy, 4),
        'recall': round(recall, 4),
        'precision': round(precision, 4),
        'f1_score': round(f1, 4)
    })

# 결과를 데이터프레임으로 출력
results_df = pd.DataFrame(results)
print("max_depth 값에 따른 성능 평가 결과:")
results_df.sort_values(by='precision', ascending=False)


max_depth 값에 따른 성능 평가 결과:


,max_depth,accuracy,recall,precision,f1_score
1,5.0,0.8380,0.6377,0.9167,0.7521
0,3.0,0.8324,0.7391,0.8095,0.7727
3,10.0,0.8212,0.7101,0.8033,0.7538
2,7.0,0.8156,0.6957,0.8000,0.7442
4,15.0,0.8156,0.7246,0.7812,0.7519
5,20.0,0.8156,0.7246,0.7812,0.7519
6,NaN,0.8156,0.7246,0.7812,0.7519


# 그리드 서치를 통한 최적의 하이퍼 파라미터 탐색

In [6]:
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 그리드 설정
param_grid = {
    'max_depth': [3, 5, 7, 10],  # 디시젼 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 내부 노드를 분할하는 데 필요한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드에 있어야 하는 최소 샘플 수
}

# GridSearchCV를 사용하여 하이퍼 파라미터 튜닝
grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),  # 디시젼 트리 분류기 사용
    param_grid=param_grid,  # 하이퍼 파라미터 그리드
    cv=5,  # 교차 검증 폴드 수
    # 클래스 비율을 균등하게 하기 위해서는 StratifiedKFold를 사용할 수 있음
    # 예: cv=StratifiedKFold(n_splits=5)
    n_jobs=-1,  # 모든 CPU 코어 사용
    scoring='f1'  # f1 score을 기준으로 평가, 가능한 문자열: 'accuracy', 'precision', 'recall', 'f1', 'roc_auc' 등
    )

# 모델 학습
grid_search.fit(X_train, y_train)

# 최적의 하이퍼 파라미터 출력
print(f'Best Parameters: {grid_search.best_params_}')

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# 성능 평가
accuracy_best = accuracy_score(y_test, y_pred_best)  # 정확도 계산
recall_best = recall_score(y_test, y_pred_best)  # 재현율 계산
precision_best = precision_score(y_test, y_pred_best)  # 정밀도 계산
f1_best = f1_score(y_test, y_pred_best)  # F1 스코어 계산

print(f'Best Decision Tree Accuracy: {round(accuracy_best, 4)}')
print(f'Best Decision Tree Recall: {round(recall_best, 4)}')
print(f'Best Decision Tree Precision: {round(precision_best, 4)}')
print(f'Best Decision Tree F1 Score: {round(f1_best, 4)}')

# 베스트 모델의 옵션값 출력
print('Best Model Options')
best_model.get_params()


Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Decision Tree Accuracy: 0.838
Best Decision Tree Recall: 0.6377
Best Decision Tree Precision: 0.9167
Best Decision Tree F1 Score: 0.7521
Best Model Options


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 42,
 'splitter': 'best'}